## Project Description

MLB Advanced Media, as stated in a job description for which I was intrigued by, was looking to develop insights into predictability of a hit based on data acquired through their Statcast tool. Statcast is a high-speed, high-accuracy device that tracks ball and player movements. 

The findings of this task would be for use by analysts and commentators during game broadcasts. The problem statement for the specfic prediction I undertook is:

Based on the ballistics of the pitch and the ball hit into play, what is the likelihood it results in a hit.

## Notebook Description

11\. Run each model through a 10-fold **cross-validation** and score each model.

- After all preprocessing steps, cross-validation confirms our findings from our original benchmarking...K Neighbors and Random Fores are the best performers for this data. Since K Neighbors scores really slowly, for the remaining notebooks only Random Forest will be used.

### Initialize packages and read in pickled data

In [1]:
# ! pip install scrapy
# ! pip install psycopg2
# ! pip install sqlalchemy
# ! pip install missingno --quiet
# ! pip install scipy

In [2]:
% run __init__.py

In [3]:
cd ..

/home/jovyan


In [4]:
df_model = pd.read_pickle('data/df_model.p')

### Resampling by Down-Sampling `no_hit` class

In [6]:
df_model_hit = df_model[df_model['hit_flag']==True]
df_model_no_hit = df_model[df_model['hit_flag']==False]
df_model_hit.shape, df_model_no_hit.shape

((41459, 88), (85593, 88))

In [7]:
df_model_no_hit_rs = df_model_no_hit.sample(df_model_hit.shape[0])

In [9]:
df_model_rs = pd.concat([df_model_hit, df_model_no_hit_rs], axis=0)

### Set up target and predictors

In [11]:
df_model_rs.drop('player_id', axis=1, inplace=True)

In [12]:
target = df_model_rs['hit_flag']
predictors = df_model_rs.drop('hit_flag', axis=1)

BoxCox requires all positive values, so I'll start this workflow by using a `MinMaxScaler` on my data

### `MinMaxScaler`

In [13]:
df_model_proc_all = predictors.copy()

In [14]:
min_max = MinMaxScaler(feature_range=(1E-10,1))

In [15]:
df_model_mm = pd.DataFrame(min_max.fit_transform(df_model_proc_all), 
                           index=df_model_proc_all.index, 
                           columns=df_model_proc_all.columns)

### Skew-Normalize Features

#### `box_cox`

In [16]:
def box_cox(predictors):
    '''Input dataframe to deskew it'''
    df_model_bc = pd.DataFrame()
    for col in predictors.columns:
        box_cox, lmbda = boxcox(predictors[col])
        df_model_bc[col] = pd.Series(box_cox)
    
    df_model_bc.set_index(predictors.index, inplace=True)
    
    return df_model_bc

In [17]:
df_model_skewnorm = box_cox(df_model_mm)

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:901: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


### Standardize Features

**`StandardScaler()`**

In [19]:
standardized = (StandardScaler().fit_transform(df_model_skewnorm))
df_standardized = pd.DataFrame(standardized, columns=df_model_skewnorm.columns, index=df_model_skewnorm.index)

### Feature Selection

#### `SelectFromModel` with L1 penalty estimator

In [22]:
sfm = SelectFromModel(LogisticRegression(penalty='l1'), threshold='mean')
sfm.fit(df_standardized, target)

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
        prefit=False, threshold='mean')

In [23]:
sfm_feats = np.where(sfm.get_support())[0]
sfm_feats

array([ 1,  2,  4,  6,  8,  9, 18])

In [24]:
columns = list(df_standardized.columns)

sfm_feats_names = []
for i in sfm_feats:
    sfm_feats_names.append(columns[i])
    
sfm_feats_names

['ev_mph',
 'dist',
 'launch_angle',
 'zone_11.0',
 'zone_13.0',
 'zone_14.0',
 'zone_unknown']

### Create dataframe with only selected features

In [25]:
df_slim = df_standardized[sfm_feats_names]

### K Neighbors Classifier - Cross Validation

In [27]:
knn = KNeighborsClassifier()
logreg = LogisticRegression()
dtree = DecisionTreeClassifier()
randforest = RandomForestClassifier()

In [28]:
scores = cross_val_score(knn, df_slim, target, cv=10, scoring='accuracy')
scores.mean()

0.77933625329280998

### Logistic Regression - Cross Validation

In [29]:
logreg_scores = cross_val_score(logreg, df_slim, target, cv=10, scoring='accuracy')
logreg_scores.mean()

0.69545838941366311

### Decision Tree Classifier - Cross Validation

In [30]:
dtree_scores = cross_val_score(dtree, df_slim, target, cv=10, scoring='accuracy')
dtree_scores.mean()

0.73982738896385658

### Random Forest Classifier - Cross Validation

In [31]:
randforest_scores = cross_val_score(randforest, df_slim, target, cv=10, scoring='accuracy')
randforest_scores.mean()

0.77405395756922979

After all preprocessing steps, cross-validation confirms our findings from our original benchmarking...K Neighbors and Random Fores are the best performers for this data. Since K Neighbors scores really slowly, for the remaining notebooks only Random Forest will be used.